필요한 도구들과 데이터를 선언하자.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

## <strong> 1. 소프트맥스 회귀 구현하기(로우-레벨) </strong>

In [6]:
print('Input shape:', x_train.shape, '/ Label shape:', y_train.shape)

Input shape: torch.Size([8, 4]) / Label shape: torch.Size([8])


In [19]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

# 총 3가지의 클래스중 하나로 구별해야 하므로 8*4->8*3으로 변환해야 한다.
W = torch.zeros((4, 3), requires_grad=True) 
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
  hypothesis = F.softmax(x_train.matmul(W)+b, dim=1)
  cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 200 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  200/1000 Cost: 0.689991
Epoch  400/1000 Cost: 0.604117
Epoch  600/1000 Cost: 0.533922
Epoch  800/1000 Cost: 0.466908
Epoch 1000/1000 Cost: 0.399962


## <strong> 2. 소프트맥스 회귀 구현하기(하이-레벨) </strong>

**F.cross_entropy()**를 사용해 비용 함수를 구현하자.

In [20]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
  z = x_train.matmul(W)+b # cross_entropy() 메서드가 softmax()를 포함하므로 선형 변환만 하면 된다.
  cost = F.cross_entropy(z, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 200 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/1000 Cost: 1.098612
Epoch  200/1000 Cost: 0.689991
Epoch  400/1000 Cost: 0.604117
Epoch  600/1000 Cost: 0.533922
Epoch  800/1000 Cost: 0.466908
Epoch 1000/1000 Cost: 0.399962


## <strong> 3. 소프트맥스 회귀 nn.Module로 구현하기 </strong>

In [22]:
model = nn.Linear(4, 3)

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  200/1000 Cost: 0.564643
Epoch  400/1000 Cost: 0.467249
Epoch  600/1000 Cost: 0.391924
Epoch  800/1000 Cost: 0.321577
Epoch 1000/1000 Cost: 0.250818


## <strong> 4. 소프트맥스 회귀 클래스로 구현하기 </strong>

In [23]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)

  def forward(self, x):
    return self.linear(x)

model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.366217
Epoch  200/1000 Cost: 0.637564
Epoch  400/1000 Cost: 0.527363
Epoch  600/1000 Cost: 0.432700
Epoch  800/1000 Cost: 0.340930
Epoch 1000/1000 Cost: 0.255350


출처: https://wikidocs.net/60575